In [ ]:
import duckdb

In [ ]:
import polars as pl

In [ ]:
import random
from random import choice
from string import ascii_lowercase, digits

import geopandas as gpd
import numpy as np
from shapely import box

from srai.constants import FEATURES_INDEX
from srai.embedders.count_embedder import CountEmbedder
from srai.h3 import ring_buffer_h3_regions_gdf
from srai.joiners.intersection_joiner import IntersectionJoiner
from srai.regionalizers.h3_regionalizer import H3Regionalizer
from srai.regionalizers.s2_regionalizer import S2Regionalizer

H3_RESOLUTION = 5
# TODO: increase after rewriting s2 regionalizer
S2_RESOLUTION = 6  # 13
H3_DISTANCE = 6

chars = ascii_lowercase + digits
columns = ["".join(choice(chars) for _ in range(8)) for _ in range(100)]
values = ["".join(choice(chars) for _ in range(8)) for _ in range(100)]

area = gpd.GeoDataFrame(geometry=[box(5.818355, 46.037418, 24.363277, 52.769854)], crs=4326)

h3_regions = H3Regionalizer(resolution=H3_RESOLUTION).transform(area)
print(f"H3 regions: {len(h3_regions)}")
buffered_h3_regions = ring_buffer_h3_regions_gdf(h3_regions, H3_DISTANCE)
print(f"Buffered H3 regions: {len(buffered_h3_regions)}")

s2_regions = S2Regionalizer(resolution=S2_RESOLUTION).transform(area)
print(f"S2 regions: {len(s2_regions)}")

data = np.full((len(s2_regions), len(columns)), None)
for i in range(len(s2_regions)):
    data[i, random.randint(0, len(columns) - 1)] = random.choice(values)

s2_regions[columns] = data
s2_regions.index.rename(FEATURES_INDEX, inplace=True)

joint = IntersectionJoiner().transform(buffered_h3_regions, s2_regions)
print(f"Joint: {len(joint)}")

count_embeddings = CountEmbedder(
    count_subcategories=True,
).transform(buffered_h3_regions, s2_regions, joint)

count_embeddings

# embeddings = ContextualCountEmbedder(
#     neighbourhood=H3Neighbourhood(),
#     neighbourhood_distance=H3_DISTANCE,
#     count_subcategories=True,
#     concatenate_vectors=False,
# ).transform(buffered_h3_regions, s2_regions, joint)

In [ ]:
count_embeddings.parquet_paths

In [ ]:
print("Dense array size:", count_embeddings.to_dataframe().values.nbytes / 1e6, "MB")

In [ ]:
columns = count_embeddings.columns

csr_rows = []
csr_cols = []
csr_values = []

print("Columns:", columns)
for batch in count_embeddings.to_duckdb(with_row_number=True).fetch_arrow_reader():
    pl_df = pl.from_arrow(batch)
    for row in pl_df.iter_rows(named=True):
        row_idx = row["row_number"] - 1  # duckdb row_number starts from 1
        for col_idx, column_name in enumerate(columns):
            val = row[column_name]
            if val > 0:
                csr_rows.append(row_idx)
                csr_cols.append(col_idx)
                csr_values.append(val)
            # if row[column_name] is not None:
            #     assert row[column_name] in values, f"Unexpected value {row[column_name]} in column {column_name}"
    # for _, row in batch:
    #     print(row)
    #     break

print("CSR size:", len(csr_values), "elements")

In [ ]:
max(csr_cols), max(csr_rows), len(data), len(columns)

In [ ]:
import scipy.sparse as sp

n_rows = count_embeddings.rows
n_cols = len(count_embeddings.columns)
coo = sp.coo_matrix((csr_values, (csr_rows, csr_cols)), shape=(n_rows, n_cols))
csr = coo.tocsr()

In [ ]:
# 2️⃣ Sparse memory (CSR)
csr_mem = csr.data.nbytes + csr.indices.nbytes + csr.indptr.nbytes
print("CSR matrix size:", csr_mem / 1e6, "MB")

# 3️⃣ Sparse memory (COO)
coo_mem = coo.data.nbytes + coo.row.nbytes + coo.col.nbytes
print("COO matrix size:", coo_mem / 1e6, "MB")

In [ ]:
import pandas as pd

df = pd.DataFrame.sparse.from_spmatrix(
    csr,
    index=count_embeddings.to_duckdb().select("region_id").fetchnumpy()["region_id"],
    columns=count_embeddings.columns,
)
df

In [ ]:
df

In [ ]:
import pyarrow as pa

arrow_sparse_csr_matrix = pa.SparseCSRMatrix.from_scipy(csr)
arrow_sparse_csr_matrix

In [ ]:
ddf = count_embeddings.to_dataframe()

print(
    "DF dense size:",
    (ddf.index.nbytes + ddf.columns.nbytes + sum(ddf[c].values.nbytes for c in ddf.columns)) / 1e6,
    "MB",
)

In [ ]:
print(f"sparse: {ddf.memory_usage().sum() / 1e6:0.2f} mb")

In [ ]:
print(f"sparse: {df.memory_usage().sum() / 1e6:0.2f} mb")

In [ ]:
ddf.join(df, rsuffix="_sparse")  # .memory_usage().sum() / 1e6

In [ ]:
print(
    "DF sparse size:",
    (df.index.nbytes + df.columns.nbytes + sum(df[c].values.nbytes for c in df.columns)) / 1e6,
    "MB",
)

In [ ]:
csr

In [ ]:
import numpy as np
import scipy.sparse as sp

rows = []
cols = []
values = []

count_embeddings.to_dataframe().values

# Imagine you iterate over your data rows
for row_idx, (region, feature_dict) in enumerate(data):
    # feature_dict = {col_idx: value, ...}, e.g. {12: 0.7, 1134: 0.2}
    for col_idx, val in feature_dict.items():
        if val != 0:
            rows.append(row_idx)
            cols.append(col_idx)
            values.append(val)

# Build sparse matrix once
n_rows = len(data)
n_cols = 5000  # number of feature columns
coo = sp.coo_matrix((values, (rows, cols)), shape=(n_rows, n_cols))
csr = coo.tocsr()

In [ ]:
rel = count_embeddings.to_duckdb()
rel.columns[1:]


cols = list(map(lambda c: f'"{c}"', rel.columns[1:4346]))

print(f"""
    SELECT x.region_id, {", ".join(cols)}
    FROM ({rel.sql_query()}) x
    """)

r = duckdb.sql(
    f"""
    SELECT region_id, array_value({", ".join(cols)}) AS embedding
    FROM read_parquet('files/pdt/CountEmbedder_20251005_132917_045489_embeddings/20251005_132917_049375.parquet')
    """
)

# rel.to_parquet('column_embeddings.parquet')
# r.to_parquet('array_embeddings.parquet')
r

In [ ]:
duckdb.read_parquet("array_embeddings.parquet").pl().cast(
    {"embedding": pl.Array(pl.Int32, shape=73)}
).head(2).select(pl.col("embedding")).sum()  # .group_by("region_id").agg(pl.col("embedding").sum())

In [ ]:
pl.read_parquet("column_embeddings.parquet")

In [ ]:
pl.read_parquet("array_embeddings.parquet")

In [ ]:
pl.read_parquet(
    "array_embeddings.parquet",
    schema={"region_id": pl.UInt64, "embedding": pl.Array(pl.Int32, shape=73)},
)

In [ ]:
pl_df = count_embeddings.to_duckdb().pl()
cols = pl_df.columns[1:]
pl_df.select(pl.str)
# pl_df

In [ ]:
pl_df

In [ ]:
duckdb.sql(
    """
    WITH arrays as (
        SELECT UNNEST([
            [1, 5, 7],
            [4, 6, 1],
            [6, 0, 7],
            [5, 3, 2]
        ]) as values
    )
    SELECT SUM(values) FROM arrays
    """
)